In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [50]:
df_tcpstream = pd.read_csv('data/bench_tcpstream.csv', delimiter=';', )
df_file = pd.read_csv('data/bench_file.csv', delimiter=';')

In [51]:
tcpstream_mean_std_time = df_tcpstream['std_total_time'].mean()
tcpstream_mean_stde2e_full_time = df_tcpstream['stde2e_total_time'].mean()
tcpstream_mean_p2m_m2m_overhead = df_tcpstream['p2m_m2m_overhead'].mean()
tcpstream_mean_stde2e_exec_time = tcpstream_mean_stde2e_full_time - tcpstream_mean_p2m_m2m_overhead
file_mean_std_time = df_file['std_total_time'].mean()
file_mean_stde2e_full_time = df_file['stde2e_total_time'].mean()
file_mean_p2m_overhead = df_file['p2m_overhead'].mean()
file_mean_stde2e_exec_time = file_mean_stde2e_full_time - file_mean_p2m_overhead

In [ ]:
# Set up the data for plotting
categories = ['File', 'TCP Stream']
std_times = [file_mean_std_time, tcpstream_mean_std_time]
stde2e_exec_times = [file_mean_stde2e_exec_time, tcpstream_mean_stde2e_exec_time]
protocoloverhead_times = [file_mean_p2m_overhead, tcpstream_mean_p2m_m2m_overhead]

# Set width of bars
bar_width = 0.35
x = np.arange(len(categories))

# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Create the bars
bars1 = ax.bar(x - bar_width/2, std_times, bar_width, label='Std Execution Time')

# Create stacked bars for StdE2E execution time and protocol overhead
bars2 = ax.bar(x + bar_width/2, stde2e_exec_times, bar_width, label='StdE2E Execution Time')
bars3 = ax.bar(x + bar_width/2, protocoloverhead_times, bar_width, 
               bottom=stde2e_exec_times, label='Protocol Overhead')

# Add labels, title and legend
ax.set_xlabel('Resource Type')
ax.set_ylabel('Execution Time (us)')
ax.set_title('Std vs StdE2E Execution with Protocol Overhead')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Add values on top of bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.0f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(bars1)
add_labels(bars2)

# Add total height labels for stacked bars
for i, (exec_time, overhead) in enumerate(zip(stde2e_exec_times, protocoloverhead_times)):
    total = exec_time + overhead
    ax.annotate(f'{total:.0f}',
                xy=(x[i] + bar_width/2, total),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.show()